In [1]:
# from pandas_agent import scrape_pandas_website
# scrape_pandas_website()

In [2]:
from sklearn_agent import load_database as sklearn_load_database
from pandas_agent import load_database as pandas_load_database
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(),override=True)

True

In [3]:
import os
pandas_database = pandas_load_database(os.environ['OPENAI_API_KEY'])

In [4]:
sklearn_database = sklearn_load_database(os.environ['OPENAI_API_KEY'])

In [5]:
from sklearn_agent import SklearnAgentChroma
from pandas_agent import PandasAgentChroma

sklearn_chroma = SklearnAgentChroma(sklearn_database)
pandas_chroma = PandasAgentChroma(pandas_database)

In [6]:
funcs = sklearn_chroma("How to do correlation analysis?")

sklearn.isotonic: Section Navigation Determine whether y is monotonically correlated with x. y is found increasing or decreasing with respect to x based on a Spearman correlation test.

sklearn.cross_decomposition: Section Navigation Canonical Correlation Analysis, also known as “Mode B” PLS. For a comparison between other cross decomposition algorithms, see Compare cross decomposition methods. Read more in the User Guide.

sklearn.feature_selection: Section Navigation Compute Pearson’s r for each features and the target. Pearson’s r is also known as the Pearson correlation coefficient. Linear model for testing the individual effect of each of many regressors. This is a scoring function to be used in a feature selection procedure, not a free standing feature selection procedure. The cross correlation between each regressor and the target is computed as: For more on usage see the User Guide. Added in version 1.0.

 sklearn.feature_selection
sklearn.feature_selection#defaults: Section Na

In [11]:
funcs[0]['full_function']

'sklearn.feature_selection.r_regression(X, y, *, center=True, force_finite=True)'

In [12]:
funcs[0]['function_calling']

"{'name': 'r_regression', 'descriptions': 'Section Navigation Compute Pearson’s r for each features and the target. Pearson’s r is also known as the Pearson correlation coefficient. Linear model for testing the individual effect of each of many regressors.\\nThis is a scoring function to be used in a feature selection procedure, not\\na free standing feature selection procedure. The cross correlation between each regressor and the target is computed\\nas: For more on usage see the User Guide. Added in version 1.0.', 'parameters': {'type': 'object', 'properties': {'X': {'type': 'array', 'description': '{array-like, sparse matrix} of shape (n_samples, n_features). The data matrix.\\n'}, 'y': {'type': 'array', 'description': 'array-like of shape (n_samples,). The target vector.\\n'}, 'center': {'type': 'boolean', 'description': 'bool, default=True. Whether or not to center the data matrix X and the target vector y.\\nBy default, X and y will be centered.\\n'}, 'force_finite': {'type': 'bo

In [20]:
from pathlib import Path

from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

executor = LocalCommandLineCodeExecutor(work_dir=work_dir)
print(
    executor.execute_code_blocks(
        code_blocks=[
            CodeBlock(language="python", code="print('Hello, World!')"),
        ]
    )
)

exit_code=0 output='Hello, World!\n' code_file='/home/athekunal/Function Calling/hierarchical-function-calling-agent/coding/tmp_code_07da107bb575cc4e02b0e1d6d99cc204.py'


In [16]:
import ast
from typing import Tuple
def sklearn_tool(query:str)->Tuple[str,dict]:
    """The sklearn tool will fetch the most relevant function definition that can answer the question that can be answered by the sklearn library. 
        It returns the full dummy function and function definition in JSON schema

    Args:
        query (str): query to the sklearn tool to fetch the most relevant function to answer the question

    Returns:
        Tuple[str,dict]: The full dummy function and function definition in JSON schema
    """
    sklearn_functions = sklearn_chroma(query)
    if len(sklearn_functions) > 0:
        return sklearn_functions[0]['full_function'],ast.literal_eval(sklearn_functions[0]['function_calling'])

In [17]:
def pandas_tool(query:str)->Tuple[str,dict]:
    """The pandas tool will fetch the most relevant function definition that can answer the question that can be answered by the pandas library. 
        It returns the full dummy function and function definition in JSON schema

    Args:
        query (str): query to the pandas tool to fetch the most relevant function to answer the question

    Returns:
        Tuple[str,dict]: The full dummy function and function definition in JSON schema
    """
    pandas_functions = pandas_chroma(query)
    print(pandas_functions)
    if len(pandas_functions) > 0:
        return pandas_functions[0]['full_function'],ast.literal_eval(pandas_functions[0]['function_calling'])

In [18]:
# full_function,func_tool = pandas_tool("How to do correlation analysis of a dataframe?")

In [19]:
# sklearn_assistant.llm_config['tools'] = func_tool

## AUTOGEN AGENT

In [42]:
import os

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

sklearn_assistant = ConversableAgent(
    name="SklearnAssistant",
    system_message="You are a helpful AI assistant that can write code. "
    "You can help with writing code using the sklearn library. You will also be given a dummy full function that you need to refer when building the function"
    "Return 'TERMINATE' when the task is done.",
    description="This agent can answer questions related to the sklearn library.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
)
pandas_assistant = ConversableAgent(
    name="PandasAssistant",
    system_message="You are a helpful AI assistant that can write code. "
    "You can help with writing code using the pandas library. You will also be given a dummy full function that you need to refer when building the function"
    "Return 'TERMINATE' when the task is done.",
    description="This agent can answer questions related to the pandas library.",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

execution_proxy = ConversableAgent(
    name="CodeExecutor",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
    code_execution_config={"executor": executor}
)

user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"]
)

In [63]:
from autogen import GroupChat, GroupChatManager

admin_proxy = ConversableAgent(
    name="CodeAdmin",
    system_message="You are a helpful AI assistnat that will decide whom to allocate the task between sklearn assistant and pandas assistant. "
    "Come up with a reasoning to decide between these two tools. The scikit-learn (sklearn assistant) library in Python is a powerful tool for machine learning. "
    " It provides simple and efficient tools for data mining and data analysis, making it easy to implement various machine learning algorithms. "
    "Key features include classification, regression, clustering, dimensionality, machine learning and preprocessing. The pandas library in Python is essential for data manipulation and analysis. "
    " It provides data structures like Series (one-dimensional) and DataFrame (two-dimensional) to handle labeled data." 
    " With pandas assistant, you can easily read and write data from various file formats (CSV, Excel, SQL, etc.), perform data cleaning, filtering, and transformation, handle missing data, and conduct complex data analysis."
    " Answer in one word only, either sklearn or pandas",
    llm_config={"config_list":[{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]}
)

In [64]:
allowed_transitions = {
    sklearn_assistant: [execution_proxy],
    pandas_assistant: [execution_proxy],
    execution_proxy: [sklearn_assistant, pandas_assistant],
}    

In [65]:
group_chat = GroupChat(
    agents = [sklearn_assistant, pandas_assistant, execution_proxy],
    messages=[],
    max_round=3,
    send_introductions=True,
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed"
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    system_message="You are a helpful AI assistnat that will decide whom to allocate the task between sklearn assistant and pandas assistant. "
    "Come up with a reasoning to decide between these two tools. The scikit-learn (sklearn assistant) library in Python is a powerful tool for machine learning. "
    " It provides simple and efficient tools for data mining and data analysis, making it easy to implement various machine learning algorithms. "
    "Key features include classification, regression, clustering, dimensionality, machine learning and preprocessing. The pandas library in Python is essential for data manipulation and analysis. "
    " It provides data structures like Series (one-dimensional) and DataFrame (two-dimensional) to handle labeled data." 
    " With pandas assistant, you can easily read and write data from various file formats (CSV, Excel, SQL, etc.), perform data cleaning, filtering, and transformation, handle missing data, and conduct complex data analysis."
    " Answer in one word only, either sklearn or pandas",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

In [66]:
# chat_result = user_proxy.initiate_chat(
#     group_chat_manager,
#     message="How to clustering?",
#     max_turns=1
# )

In [67]:
reply = admin_proxy.generate_reply(messages=[{"content": "How to do clustering?", "role": "user"}])
print(reply)


>>>>>>>> USING AUTO REPLY...
sklearn


In [99]:
question = "How to do clustering with KMeans?"
# while True:
reply = admin_proxy.generate_reply(messages=[{"content": question , "role": "user"}])

if reply not in ['sklearn', 'pandas']:
    question_with_error = f"Please output sklearn or pandas based on the question. Don't output {reply}\n Question: {question}"
    reply = admin_proxy.generate_reply(messages=[{"content": question_with_error , "role": "user"}])

if reply == "sklearn":
    sklearn_full_function,sklearn_function = sklearn_tool(question)
elif reply == "pandas":
    pandas_full_function,pandas_function = pandas_tool(question)



>>>>>>>> USING AUTO REPLY...
sklearn.cluster: Section Navigation Perform K-means clustering algorithm. Read more in the User Guide.

sklearn.cluster: Section Navigation K-Means clustering. Read more in the User Guide.

sklearn.cluster: Section Navigation Mini-Batch K-Means clustering. Read more in the User Guide.

 sklearn.cluster
sklearn.cluster#defaults: Section Navigation Perform K-means clustering algorithm. Read more in the User Guide.

sklearn.cluster#defaults: Section Navigation K-Means clustering. Read more in the User Guide.

sklearn.cluster#defaults: Section Navigation Mini-Batch K-Means clustering. Read more in the User Guide.


sklearn.cluster#defaults
{'function_trail': {'$eq': 'sklearn.cluster-->defaults'}}


In [100]:
sklearn_function_tool = [
    {'type':'function',
     'function':sklearn_function
    }
]

In [101]:
sklearn_function_tool

[{'type': 'function',
  'function': {'name': 'k_means',
   'descriptions': 'Section Navigation Perform K-means clustering algorithm. Read more in the User Guide.',
   'parameters': {'type': 'object',
    'properties': {'X': {'type': 'array',
      'description': '{array-like, sparse matrix} of shape (n_samples, n_features). The observations to cluster. It must be noted that the data\nwill be converted to C ordering, which will cause a memory copy\nif the given data is not C-contiguous.\n'},
     'n_clusters': {'type': 'integer',
      'description': 'int. The number of clusters to form as well as the number of\ncentroids to generate.\n'},
     'sample_weight': {'type': 'array',
      'description': 'array-like of shape (n_samples,), default=None. The weights for each observation in X. If None, all observations\nare assigned equal weight. sample_weight is not used during\ninitialization if init is a callable or a user provided array.\n'},
     'init': {'type': 'object',
      'descripti

In [102]:
sklearn_assistant.llm_config['tools'] = sklearn_function_tool

In [103]:
sklearn_assistant.llm_config['tools']

[{'type': 'function',
  'function': {'name': 'k_means',
   'descriptions': 'Section Navigation Perform K-means clustering algorithm. Read more in the User Guide.',
   'parameters': {'type': 'object',
    'properties': {'X': {'type': 'array',
      'description': '{array-like, sparse matrix} of shape (n_samples, n_features). The observations to cluster. It must be noted that the data\nwill be converted to C ordering, which will cause a memory copy\nif the given data is not C-contiguous.\n'},
     'n_clusters': {'type': 'integer',
      'description': 'int. The number of clusters to form as well as the number of\ncentroids to generate.\n'},
     'sample_weight': {'type': 'array',
      'description': 'array-like of shape (n_samples,), default=None. The weights for each observation in X. If None, all observations\nare assigned equal weight. sample_weight is not used during\ninitialization if init is a callable or a user provided array.\n'},
     'init': {'type': 'object',
      'descripti

In [120]:
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

chat_result = user_proxy.initiate_chat(sklearn_assistant, message=question,max_turns=2)

User (to SklearnAssistant):

How to do clustering with KMeans?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
SklearnAssistant (to User):

To perform clustering with KMeans using the sklearn library in Python, you can follow the general steps below:

1. Import the necessary libraries:
```python
from sklearn.cluster import KMeans
```

2. Create a KMeans object with the desired number of clusters (n_clusters) and other parameters if needed:
```python
kmeans = KMeans(n_clusters=3, random_state=0)
```

3. Fit the KMeans model to your data:
```python
kmeans.fit(data)
```

4. Get the predicted clusters for your data points:
```python
predicted_clusters = kmeans.predict(data)
```

5. Get the cluster centers:
```python
cluster_centers = kmeans.cluster_centers_
```

6. You can also assign new data points to the existing clusters using the `predict` method:
```python
new_data = [[1, 2], [3, 4]]
predicted_new_data_clusters = kmeans.